In [362]:
import numpy as np
import math

In [363]:
def check_on_curve(curve, x, y):  #  curve modulo n defined by a, b and modulo n
    if x == "inf" and y == "inf":  #  point at infinity is always in the group
        return 1
    if (y ** 2) % curve[2] == (x ** 3 + curve[0] * x + curve[1]) % curve[2]:
        return 1
    return 0

In [364]:
c = [5 , -5, 41]  # curve that has a = 5, b = -5 and is mod 41
print(check_on_curve(c, 0, 0))
print(check_on_curve(c, 1, 1))

0
1


In [365]:
def inverse_mod_n(a, n):
    if np.gcd(a, n) != 1:
        return 0  # return 0 if no inverse is possible i.e the gcd 
    return pow(a, -1, n)

In [366]:
print(inverse_mod_n(1801, 2713))
print(inverse_mod_n(3, 7))
print(inverse_mod_n(14, 35))
print(inverse_mod_n(-3, 5))

2359
5
0
3


In [367]:
def add_on_curve(curve, x1, y1, x2, y2):  #  curve[0] is a, curve[1] is b and curve[2] is n
    if check_on_curve(curve, x1, y1) == False or check_on_curve(curve, x2, y2) == False:
        return "error, one of the points is not on the curve"
    if -1 * y1 == y2 and x1 == x2:
        return ["inf", "inf"]  #  if we add P(x, y) to P(x, -y) we obtain the point at infinity
    if x1 == "inf" and y1 == "inf":
        return [x2, y2]
    if x2 == "inf" and y2 == "inf":  #  the point at infinity is the identity element to addition on curve
        return [x1, y1]

    n = curve[2]
    inv = inverse_mod_n(x2 - x1, n)
    if inv == 0:  #  the inverse fails, so the gcd must be > 1 and produce some common factor
        return "no_inv_for_add_check_gcd" ###########
    lam = ((y2 - y1) * inv) % n
    x3 = (lam ** 2 - x1 - x2) % n
    y3 = (lam * (x1 - x3) - y1) % n
    if n < 2 * y3:
        y3 = y3 - n  #  use the residue class with smallest absolute value to simplify the calculation for example 32 mod 41 is -9 mod 41
    return [x3, y3]

In [368]:
print(add_on_curve(c, 1, 1, 3, 18))
print(add_on_curve(c, 1, 1, 1, -1))
print(add_on_curve(c, 0, 0, 3, 18))
print(add_on_curve(c, "inf", "inf", 3, 18))


[17, -14]
['inf', 'inf']
error, one of the points is not on the curve
[3, 18]


In [369]:
def double_on_curve(curve, x, y):
    if check_on_curve(curve, x, y) == 0:
        return "error, the point is not on curve"
    a = curve[0]
    n = curve[2]
    inv2y = inverse_mod_n(int(2 * y), n)
    if inv2y == 0:
        return "no_inv_for_double_check_gcd"
    lam = (int((3 * (x ** 2) + a) * inv2y)) % n
    xf = (lam ** 2 - 2 * x) % n
    yf = (lam * (x - xf) - y) % n
    if n < 2 * yf:
        yf = yf - n
    return [xf, yf]

In [370]:
print(double_on_curve(c, 3, 18))

[12, 15]


In [371]:
def scale_on_curve(curve, x, y, k):  #  find point kP where P(x, y) on curve mod n
    n = curve[2]
    bink = f'{k:00b}'
    bink = bink[::-1]  #  turn into binary number to perform modular doubling
    to_double = [x, y]  # start from P, then double it and at each step add the results to get kP
    found_first_power = 0 # found first power of 2 multiple of to add up to k
    

    for i in range(1, len(bink)):
        if int(bink[i]) == 1:
            if found_first_power == 0:
                kp = to_double  # initialise the list of powers to be considered
                found_first_power = 1
            else:
                if add_on_curve(curve, kp[0], kp[1], to_double[0], to_double[1]) == "no_inv_for_add_check_gcd":  #  there is no inverse of x2 - x1 mod n so we take x2 - x1 as our candidate to compute gcd
                    return (to_double[0] - kp[0]) % n
                
                kp = add_on_curve(curve, kp[0], kp[1], to_double[0], to_double[1])

        if double_on_curve(curve, to_double[0], to_double[1]) == "no_inv_for_double_check_gcd":  #  there is no inverse of 2 * y mod n in doubling so we choose 2 * y
            return ( 2 * to_double[1]) % n
        to_double = double_on_curve(curve, to_double[0], to_double[1])
    return -1

    


In [372]:
a, b, n = 5, -5, 482431  #  a, b from the elliptic curve and the modulus i.e the number we want to factorise 
xp, yp = 1, 1  #  point P to scale
k = math.factorial(6)  #  artitrary k times to scale P, good choices are highly composite numbers
d = scale_on_curve([a, b, n], 1, 1, k)
print(f'input for gcd test: {d}')
print(f'a factor of {n} is: {np.gcd(d, n)}\n')

a, b, n = 5, -5, 1000003 * 9999991 * 21319 #  1000003, 9999991 and 21319 are prime
xp, yp = 1, 1
k = math.factorial(150)
d = scale_on_curve([a, b, n], 1, 1, k)
print(f'input for gcd test: {d}')
print(f'a factor of {n} is: {np.gcd(d, n)}')

input for gcd test: 287255
a factor of 482431 is: 787

input for gcd test: 94810373970521773
a factor of 213190447698424387 is: 21319
